In [2]:
"""
import the necessary libraries
"""
import pandas as pd

In [3]:
shrooms = pd.read_table('agaricus-lepiota.data', delimiter=',', header=None)

In [4]:
colnames = [
    'poisonous', 'cap_shape', 'cap_surface', 'cap_color', 'bruised', 'odor',
    'gill_attachment', 'gill_spacing', 'gill_size', 'gill_color', 
    'stalk_shape', 'stalk_root', 'stalk_surface_above_ring',
    'stalk_surface_below_ring', 'stalk_color_above_ring',
    'stalk_color_below_ring', 'veil_type', 'veil_color', 'ring_number',
    'ring_type', 'spore_print_color', 'population', 'habitat'
]
shrooms.columns = colnames

In [5]:
shrooms.head()

,poisonous,cap_shape,cap_surface,cap_color,bruised,odor,gill_attachment,gill_spacing,gill_size,gill_color,...,stalk_surface_below_ring,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [6]:
shrooms.dtypes

poisonous                   object
cap_shape                   object
cap_surface                 object
cap_color                   object
bruised                     object
odor                        object
gill_attachment             object
gill_spacing                object
gill_size                   object
gill_color                  object
stalk_shape                 object
stalk_root                  object
stalk_surface_above_ring    object
stalk_surface_below_ring    object
stalk_color_above_ring      object
stalk_color_below_ring      object
veil_type                   object
veil_color                  object
ring_number                 object
ring_type                   object
spore_print_color           object
population                  object
habitat                     object
dtype: object

In [11]:
shrooms.duplicated().sum()

0